In [4]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [11]:
import os.path
from pathlib import Path

import json
import pyam
import scmdata
import pandas as pd
import numpy as np
import tqdm.autonotebook as tqdman
from pymagicc.io import MAGICCData

import utils

## plotting packages
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

<IPython.core.display.Javascript object>

In [12]:
# number of scenarios
NR_SCEN = 200

<IPython.core.display.Javascript object>

In [34]:
SRM_START_YEAR = 2030
ENSEMBLE = list(range(0, 600 + 1))
YEARS = range(2030, 2500 + 1)
ID = "small_selection"

SCENARIOS_min = [0, 100, 200, 300]
SCENARIOS_max = [99, 199, 299, 355]

SCENARIOS_min = [0, 10, 51, 101, 151]
SCENARIOS_max = [9, 50, 100, 150, 200]

<IPython.core.display.Javascript object>

In [35]:
OUTPUT_FILE = (
    utils.DATA_DIR
    / "interim"
    / "magicc-output"
    / f"magicc-output_{ID}_{SCENARIOS_min[0]}_{SCENARIOS_max[0]}_totalCDR.csv"
)
OUTPUT_FILE.parent.parent.mkdir(exist_ok=True)
OUTPUT_FILE.parent.mkdir(exist_ok=True)

<IPython.core.display.Javascript object>

#### Extended scenario temperature data from 021

In [36]:
datasets = []

for min_, max_ in tqdman.tqdm(zip(SCENARIOS_min, SCENARIOS_max)):
    DATA_FILE = Path(
        utils.DATA_DIR
        / "interim"
        / "magicc-output"
        / f"magicc-output_{ID}_{min_}_{max_}.csv"
    )
    datasets.append(scmdata.ScmRun(str(DATA_FILE)))
# data_scen = datasets[0].append(datasets[1])
# data_scen = data_scen.append(datasets[2])
data_scen = scmdata.run_append(datasets)
# data_scen = datasets[0]
data_scen["ms"] = data_scen["model"] + data_scen["scenario"]
ms_list = data_scen.get_unique_meta("ms")
len(ms_list)

0it [00:00, ?it/s]

196

<IPython.core.display.Javascript object>

In [37]:
len(ms_list)

196

<IPython.core.display.Javascript object>

In [38]:
# use only a few scenarios and ensemble_members
data_scen_sel = data_scen.filter(ms=ms_list[:NR_SCEN], ensemble_member=ENSEMBLE)

# round to 3 decimals
data_scen_sel = data_scen_sel.round(3)

### export and import as pandas
data_scen_sel.filter(variable="*Temperature*").to_csv(
    str(Path(utils.DATA_DIR / "interim" / "layover" / f"{ID}_T_filtered.csv"))
)
data_scen_sel.to_csv(
    str(Path(utils.DATA_DIR / "interim" / "layover" / f"{ID}_filtered.csv"))
)

/data/ubuntu-znicholls/miniconda3/envs/baur-et-al-srm-commitment/lib/python3.9/site-packages/scmdata/run.py:2246: UserWarning: There are small values which may be truncated during rounding. Either increase the numberof decimals or convert the units of the timeseries so that the quantities are larger.
  warnings.warn(


<IPython.core.display.Javascript object>

In [39]:
data_scen_T_pd = (
    scmdata.ScmRun(
        str(Path(utils.DATA_DIR / "interim" / "layover" / f"{ID}_T_filtered.csv"))
    )
    .timeseries(time_axis="year")
    .reset_index()
)
data_scen_pd = pd.read_csv(
    str(Path(utils.DATA_DIR / "interim" / "layover" / f"{ID}_filtered.csv"))
)

<IPython.core.display.Javascript object>

#### Extended scenario emissions data from 012

In [40]:
# load in as SCM to easily combine fossil and AFOLU
SOURCE_DATA = Path(
    utils.DATA_DIR
    / "interim"
    / "ar6-wg3-extended"
    / f"wg3-timeseries-extended_{ID}.csv"
)

<IPython.core.display.Javascript object>

# Add total CO2

In [ ]:
our_set = scmdata.ScmRun(str(SOURCE_DATA)).filter(
    variable=["*AFOLU*", "*Energy*"], ms=ms_list[:NR_SCEN]
)

our_set_co2_total = our_set.filter(variable="*CO2*").process_over(
    "variable",
    "sum",
    op_cols={"variable": "AR6 climate diagnostics|Infilled|Emissions|CO2"},
    as_run=True,
)

In [41]:
emissions_data = (
    scmdata.ScmRun(emissions_data).timeseries(time_axis="year").reset_index()
)
emissions_data.head()

time,max_CDR,model,ms,region,scenario,total_CDR,total_emissions,unit,variable,2015,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,-8534.394684,COFFEE 1.1,COFFEE 1.1EN_INDCi2030_500,World,EN_INDCi2030_500,-2.744009e+06,224839.900719,Mt CO2/yr,AR6 climate diagnostics|Infilled|Emissions|CO2,39152.7263,...,-8534.394684,-8534.394684,-8534.394684,-8534.394684,-8534.394684,-8534.394684,-8534.394684,-8534.394684,-8534.394684,-8534.394684
1,-5055.616650,COFFEE 1.1,COFFEE 1.1EN_INDCi2030_600,World,EN_INDCi2030_600,-1.960631e+06,288411.065216,Mt CO2/yr,AR6 climate diagnostics|Infilled|Emissions|CO2,39152.7263,...,-5055.616650,-5055.616650,-5055.616650,-5055.616650,-5055.616650,-5055.616650,-5055.616650,-5055.616650,-5055.616650,-5055.616650
2,-7870.110129,POLES ADVANCE,POLES ADVANCEADVANCE_2030_1.5C-2100,World,ADVANCE_2030_1.5C-2100,-5.373515e+06,321312.950414,Mt CO2/yr,AR6 climate diagnostics|Infilled|Emissions|CO2,39152.7263,...,-7870.110129,-7870.110129,-7870.110129,-7870.110129,-7870.110129,-7870.110129,-7870.110129,-7870.110129,-7870.110129,-7870.110129
3,-32376.733249,COFFEE 1.1,COFFEE 1.1EN_INDCi2030_600_NDCp,World,EN_INDCi2030_600_NDCp,-6.883314e+06,325587.181056,Mt CO2/yr,AR6 climate diagnostics|Infilled|Emissions|CO2,39152.7263,...,-32376.733249,-32376.733249,-32376.733249,-32376.733249,-32376.733249,-32376.733249,-32376.733249,-32376.733249,-32376.733249,-32376.733249
4,-10762.243952,GEM-E3_V2021,GEM-E3_V2021EN_INDCi2030_600f,World,EN_INDCi2030_600f,-4.040064e+06,397215.836129,Mt CO2/yr,AR6 climate diagnostics|Infilled|Emissions|CO2,39152.7263,...,-10762.243952,-10762.243952,-10762.243952,-10762.243952,-10762.243952,-10762.243952,-10762.243952,-10762.243952,-10762.243952,-10762.243952


<IPython.core.display.Javascript object>

### Bringing temperature and cumulative values together

In [42]:
def find_closest_value(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

<IPython.core.display.Javascript object>

In [44]:
col_list = list(range(2015, 2500 + 1))
data_scen_T_pd["year_15T"] = 0
data_scen_T_pd["total_CDR_at_15"] = 0

for i in tqdman.tqdm(range(0, len(data_scen_T_pd))):

    current_ms = data_scen_T_pd.iloc[i].ms

    ###### GET CUMULATIVE CDR
    # calculate cum CDR
    emissions_data_ms = emissions_data[emissions_data["ms"] == current_ms]

    # set all positive values 0
    total_CDR_data = (
        emissions_data_ms[
            emissions_data_ms["variable"]
            == "AR6 climate diagnostics|Infilled|Emissions|CO2"
        ][col_list]
        .apply(lambda x: x[x < 0])
        .iloc[:, 15:]
        .cumsum(axis=1)
    ).fillna(0)

    ###### FIND YEAR OF 1.5
    closest_value = find_closest_value(data_scen_T_pd.iloc[i, 292:], 1.5)
    # transpose
    j = i + 1
    long_df = data_scen_T_pd.iloc[i:j, 292:].T
    # add new column with bool
    long_df["BoolCol"] = long_df[i] == closest_value
    # get year
    year_15T = long_df.index[long_df["BoolCol"] == True].tolist()

    # if clause to check for all pathways that never reach closer to 1.5 than they were in 2030 -> replace year with 2500
    # if year_15T[0] in list(range(2030, 2050)) and data_scen_T_pd.iloc[i][2500] > 1.5:
    if data_scen_T_pd.iloc[i][2500] > 1.5:
        # add year to column
        data_scen_T_pd["year_15T"].iloc[i] = 2500
        # look at cumulative CDR value in that year and add it as a new column
        total_CDR_value_at_15 = total_CDR_data[2500]
    else:
        # add year to column
        data_scen_T_pd["year_15T"].iloc[i] = year_15T[0]
        # look at cumulative CDR value in that year and add it as a new column
        total_CDR_value_at_15 = total_CDR_data[year_15T[0]]

    # Exception for pathways that throw errors (to check!)
    #    data_scen_T_pd["total_CDR_at_15"].iloc[i] = total_CDR_value_at_15
    try:
        data_scen_T_pd["total_CDR_at_15"].iloc[i] = total_CDR_value_at_15
    except:
        data_scen_T_pd["total_CDR_at_15"].iloc[i] = 1

  0%|          | 0/117600 [00:00<?, ?it/s]


KeyboardInterrupt



<IPython.core.display.Javascript object>

### Add back into original dataset
If error appears it's likely due to different numbers of ensemble members (check ENSEMBLE and number of ens in data_scen_T_pd)

In [ ]:
data_scen_pd["total_CDR_at_15"] = 0
data_scen_pd["year_15T"] = 0


for ms_ in tqdman.tqdm(ms_list):
    ms_filt_T = data_scen_T_pd[data_scen_T_pd["ms"] == ms_]
    ms_filt = data_scen_pd[data_scen_pd["ms"] == ms_]
    for ens in ENSEMBLE[:-1]:
        total_CDR_at_15 = ms_filt_T.loc[ms_filt_T["ensemble_member"] == ens,
            "total_CDR_at_15"
        ]
        year_15T = ms_filt_T.loc[ms_filt_T["ensemble_member"] == ens,
            "year_15T"
        ]
        ## add value to original dataset
        data_scen_pd.loc[(data_scen_pd["ms"] == ms_) & (data_scen_pd["ensemble_member"] == ens), "total_CDR_at_15"] = total_CDR_at_15.item()
        data_scen_pd.loc[(data_scen_pd["ms"] == ms_) & (data_scen_pd["ensemble_member"] == ens), "year_15T"] = year_15T.item()
        


### Save

In [ ]:
data_scen_pd.to_csv(str(OUTPUT_FILE))